<H1>Console Collection</H1>
<hr>
<p>Our original data set consists of a bunch of a values that I copied and pasted about a half a year ago. Using python, hopefully this collection technique will be more efficient.</p>

<p>Collection goals:
<ul>
    <li><strike>Playstation</strike></li>
    <li><strike>PS2</strike></li>
    <li><strike>Wii</strike></li>
    <li><strike>Dreamcast</strike></li>
    <li><strike>Genesis</strike></li>
    <li><strike>3DS</strike></li>
</ul>
</p>
<br>
<p>To scrape games for most consoles, we decided to use a popular game archive site. One issue we encountered is that some cells like Player and Year were the same size, so in consoles that included player counts we had to modify our code.</p>

In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import csv

def siteScraperNP(website,console):
    site = website
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)

    links = []
    for link in soup.findAll('a', attrs={'href': re.compile("/vault/")}):
        links.append(link)

    #getting link of first page to index
    j = 0
    scrapeLinkNum = -1
    for link in links:
        if re.search("#",str(link)):
            #print("links" + "[%d]"%(j))
            scrapeLinkNum = j
        j+=1

    #adding the alphabet
    scrapeLinks = []
    reQuote = re.compile(r'([\"\'])(?:(?=(\\?))\2.)*?\1')
    for i in range(27):
        x = str(links[scrapeLinkNum + i])
        y = reQuote.search(x)
        x = y.group()
        x = x.replace("\"","")
        x = x.replace("&amp;","&")
        x = "website" + x
        scrapeLinks.append(x)
    
    #getting the titles
    titles=[]
    for i in scrapeLinks:
        site = i
        hdr = {'User-Agent': 'Mozilla/5.0'}
        req = Request(site,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page)

        for t in soup.findAll('tr',{'class':['odd', 'even']}):
            titles.append(t)
            
    consoleName = console
    j = 0
    games = []
    for i in titles:
        a = titles[j].findChild("a").contents[0]
        try:
            b = titles[j].findChild("td",attrs={'style':'width:8%; text-align:center'}).contents[0]
        except:
            b = 0
        try:
            c = titles[j].findChild("td",attrs={'style':'width:20%; text-align:center; font-size:10pt'}).contents[0]
        except:
            c = ""

        x = [consoleName,a,b,c]
        games.append(x)
        j+=1
    return games

def siteScraperP(website,console):
    site = website
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)

    links = []
    for link in soup.findAll('a', attrs={'href': re.compile("/vault/")}):
        links.append(link)

    #getting link of first page to index
    j = 0
    scrapeLinkNum = -1
    for link in links:
        if re.search("#",str(link)):
            #print("links" + "[%d]"%(j))
            scrapeLinkNum = j
        j+=1

    #adding the alphabet
    scrapeLinks = []
    reQuote = re.compile(r'([\"\'])(?:(?=(\\?))\2.)*?\1')
    for i in range(27):
        x = str(links[scrapeLinkNum + i])
        y = reQuote.search(x)
        x = y.group()
        x = x.replace("\"","")
        x = x.replace("&amp;","&")
        x = "website" + x
        scrapeLinks.append(x)
    
    #getting the titles
    titles=[]
    for i in scrapeLinks:
        site = i
        hdr = {'User-Agent': 'Mozilla/5.0'}
        req = Request(site,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page)

        for t in soup.findAll('tr',{'class':['odd', 'even']}):
            titles.append(t)
            
    consoleName = console
    j = 0
    games = []
    for i in titles:
        a = titles[j].findChild("a").contents[0]
        try:
            children = titles[j].findAll("td",attrs={'style':'width:8%; text-align:center'})
            b = []
            for child in children:
                b.append(child.contents[0])
            b = b[1]
        except:
            b = 0
        try:
            c = titles[j].findChild("td",attrs={'style':'width:20%; text-align:center; font-size:10pt'}).contents[0]
        except:
            c = ""
            
        x = [consoleName,a,b,c]
        games.append(x)
        j+=1
    return games

In [2]:
# P = players , NP = No players

ps1games = siteScraperP("website","PlayStation")
wiiGames = siteScraperNP("website","Wii")
dreamcast = siteScraperP("website","Dreamcast")
genesis = siteScraperP("website","Genesis")

In [148]:
def outCSV(listname,filename):
    with open(filename + ".csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(listname)

outCSV(ps1games,"ps1")
outCSV(wiiGames,"wii")
outCSV(dreamcast,"dreamcast")
outCSV(genesis,"genesis")

<p>The website doesn't have any entries for 3DS, so we'll need to use wikipedia.<br><br>Thankfully, wikipedia had a large sheet of DS games. Below we scrape the list, drop columns we don't need, and save it as a CSV.</p>

In [209]:
import pandas as pd

df = pd.read_html("https://en.wikipedia.org/wiki/List_of_Nintendo_3DS_games",header=[1])[0]

In [210]:
a = df.drop([df.columns[1] , df.columns[3], df.columns[5], df.columns[6], df.columns[7]] ,  axis='columns')
a = a.rename(columns={"North America": "release"})
df = a[~a.release.str.contains("Unreleased", na=False)]

In [216]:
df['release2'] = df['release'].astype(str).str[-4:]

<ipython-input-216-aa00a304cbd2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release2'] = df['release'].astype(str).str[-4:]


,Title,Publisher(s),release,release2
0,10-in-1: Arcade Collection,Gamelion Studios,"August 15, 2013",2013
3,1001 Spikes,Nicalis,"June 3, 2014",2014
4,101 DinoPets 3D,Selectsoft,"January 10, 2013",2013
5,101 Penguin Pets 3D,Selectsoft,"October 17, 2013",2013
6,101 Pony Pets 3D,Selectsoft,"July 24, 2014",2014
...,...,...,...,...
1340,Zeus Quest Remastered Anagenissis of Gaia,Crazysoft,"July 5, 2018",2018
1341,Zig Zag Go,RCMADIAX,"January 25, 2018",2018
1342,Zombie Incident,CoderChild,"March 5, 2015",2015
1343,Zombie Slayer Diox,JP: CyberfrontWW: UFO Interactive Games,"March 22, 2012",2012


In [219]:
df = df.drop(df.columns[2], axis='columns')
df = df.rename(columns={"release2": "release"})

In [222]:
df.to_csv('3DS.csv', index=False, header=False)